In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [8]:
IMG_WIDTH = 620
IMG_HEIGHT = 877
IMG_CHN = 3
NUM_F_POINTS = 5000
NUM_MATCHES = 2000
BBOX_LENGTH = 21     

In [9]:
# Batch processing of cv2 non-learning based method
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

def extract_features(ref_image, sns_image):
    # Create ORB detector with 5000 features. 

    ref_image = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY) 
    sns_image = cv2.cvtColor(sns_image, cv2.COLOR_BGR2GRAY) 
    orb_detector = cv2.ORB_create(5000) 
    kp1, d1 = orb_detector.detectAndCompute(ref_image, None) 
    kp2, d2 = orb_detector.detectAndCompute(sns_image, None) 
    return [kp1,kp2,d1,d2]

def extract_feature_batch(refs, sns):
    output = [[],[],[],[]]
    for i in range(refs.shape[0]):
        out = extract_features(refs[i], sns[i])
        for j in range(4):
            output[j].append(out[j])
    
    return output

def batch_match(d1s, d2s):
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    matches = []

    for i in range(len(d1s)):
        matches.append(matcher.match(d1s[i], d2s[i]))
    return matches

def validate_match(matches):
    for i in range(len(matches)):
        matches[i].sort(key = lambda x: x.distance)
        matches[i] = matches[i][:int(len(matches)*60)]
    return matches

def calc_homographies(kp1s, kp2s, matches):
    # Define empty matrices of shape no_of_matches * 2. 
    homographies = []
    
    temp = matches
    matches = []
    for i in range(len(temp)):
        if temp[i] != []:
            matches.append(list(filter(None, temp[i])))
            
    matches = validate_match(matches)
    
    for i in range(len(matches)):
        p1 = np.zeros((len(matches[i]), 2)) 
        p2 = np.zeros((len(matches[i]), 2)) 
        for j in range(len(matches[i])):
            p1[j, :] = kp1s[i][matches[i][j].queryIdx].pt 
            p2[j, :] = kp2s[i][matches[i][j].trainIdx].pt 
        homography, _ = cv2.findHomography(p1, p2, cv2.RANSAC) 
        homographies.append(homography)
    return homographies

def register_images(sns_imgs, homographies, img_size = (IMG_WIDTH,IMG_HEIGHT), save = False):
    # Use this matrix to transform the 
    # colored image wrt the reference image. 
    transformed_imgs = []
    for i in range(sns_imgs.shape[0]):
        transformed_img = cv2.warpPerspective(sns_imgs[i], 
                            homographies[i], img_size) 
        if save: 
            cv2.imwrite('aligned_{}.jpg'.format(i), transformed_img) 
        
        transformed_imgs.append(transformed_img)
    return transformed_imgs

def visualize_matches(ref_imgs, sns_imgs, kp1s, kp2s, matches):
    for i in range(ref_imgs.shape[0]):
        imMatches = cv2.drawMatches(ref_imgs[i], kp1s[i], sns_imgs[i], kp2s[i], matches[i], None)
        cv2.imwrite("matches_{}.jpg".format(i), imMatches)
    
    
def get_alignment_matrix(kprs, kpss, drs, dss):
    '''
    Inputs
        kprs: F keypoints for each reference(query) image of shape N*F*3 with X,Y,Size
        kpss: F keypoints for each sensed(train) image of shape N*F*3 with X,Y,Size
        drs: F feature descriptors for each reference(query) image of shape N*F*32 
        dss: F feature descriptors for each sensed(train) image of shape N*F*32 
    Output
        aligned feature points and correlated distance of size N*f*7 X1,Y1,Size1, X2, Y2, Size2, Distance
    '''

    alignment_matrices = None
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    for i in range(kprs.shape[0]):
        results = matcher.match(d1s[i], d2s[i])
        aligned = []
        for r in results:
            if len(aligned) >= NUM_MATCHES: break
            temp = np.concatenate((kprs[i][r.queryIdx], kpss[i][r.trainIdx]))
            aligned.append(np.concatenate((temp, [r.distance])))
            
        while len(aligned) < NUM_MATCHES:
            aligned.append([0,0,0,0,0,0,0])
            
        aligned = np.array([aligned])
        if alignment_matrices is None:
            alignment_matrices = aligned
        else:
            alignment_matrices = np.vstack((alignment_matrices, aligned))
            
    #sample alignment_matrix
    alignment_matrices = np.rint(alignment_matrices).astype(np.uint32)
    return alignment_matrices
    
    
def test_cv2_batch(dl):
    refs, sns = dl.load_image()
    kp1s, kp2s, d1s,d2s = extract_feature_batch(refs,sns)
    matches = batch_match(d1s, d2s)
    
    visualize_matches(refs, sns, kp1s, kp2s, matches)
    homos = calc_homographies(kp1s, kp2s, matches)
    imgs = register_images(sns, homos)
    for i in range(6):
            cv2.imwrite("registered_cv2{}.jpg".format(i), imgs[i])
        

    

In [10]:
NUM_F_POINTS = 2000

def extract_feature_coordinates(ref_image, sns_image):
    # Create ORB detector with 5000 features. 
    ref_image = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY) 
    sns_image = cv2.cvtColor(sns_image, cv2.COLOR_BGR2GRAY) 
    orb_detector = cv2.ORB_create(NUM_F_POINTS) 
    kp1, d1 = orb_detector.detectAndCompute(ref_image, None) 
    kp2, d2 = orb_detector.detectAndCompute(sns_image, None) 
    kp1_np, kp2_np = [], []
    for i in range(NUM_F_POINTS):
        if i < len(kp1):
            kp1_np.append([kp1[i].pt[0],kp1[i].pt[1], kp1[i].size ] )
        else:
            kp1_np.append([0,0,0])
            d1 = np.vstack((d1, [np.zeros(32, dtype = np.uint8 )]))
            
        if i < len(kp2):
            kp2_np.append([kp2[i].pt[0],kp2[i].pt[1], kp2[i].size ] )
        else:
            kp2_np.append([0,0,0])
            d2 = np.vstack((d2, [np.zeros(32,dtype = np.uint8 )]))
        
    kp1_np, kp2_np = np.array(kp1_np) , np.array(kp2_np)

    return [kp1_np, kp2_np, d1, d2]

def extract_feature_coor_batch(refs, sns):
    output = []
    for i in range(refs.shape[0]):
        out = extract_feature_coordinates(refs[i], sns[i])
        for j in range(4):
            if len(output) < 4:
                output.append(np.expand_dims(out[j], axis=0))
            else: 
                output[j] = np.vstack( (output[j],np.expand_dims(out[j], axis=0)) )

    return output


def extract_match_patches(ref_imgs, sns_imgs, kprs, kpss, drs, dss):
    '''
    output: N * NUM_MATCHES * 2 * PATCH_H * PATCH_W * CHN Example:(6, 500, 2, 6, 6, 3)
    '''
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    patches, matches = [], []
    for i in range(kprs.shape[0]):
        results = matcher.match(drs[i], dss[i])

        patch = []
        match = []
        for r in results:
            pair = []
            match_p = []
            if len(patch) >= NUM_MATCHES: break
            x, y, dis = kprs[i][r.queryIdx].astype(np.uint32)
            if  x-(BBOX_LENGTH-1) > 0 and y-(BBOX_LENGTH-1) > 0 \
                and x+(BBOX_LENGTH-1)/2 < IMG_WIDTH and y+(BBOX_LENGTH-1)/2 < IMG_HEIGHT:
                pair.append(ref_imgs[i][y-int((BBOX_LENGTH-1)/2):y+int((BBOX_LENGTH-1)/2)\
                                       ,x-int((BBOX_LENGTH-1)/2):x+int((BBOX_LENGTH-1)/2)])
                
            x, y, dis = kpss[i][r.trainIdx].astype(np.uint32)
            if  x-(BBOX_LENGTH-1) > 0 and y-(BBOX_LENGTH-1) > 0 \
                and x+(BBOX_LENGTH-1)/2 < IMG_WIDTH and y+(BBOX_LENGTH-1)/2 < IMG_HEIGHT:
                pair.append(sns_imgs[i][y-int((BBOX_LENGTH-1)/2):y+int((BBOX_LENGTH-1)/2)\
                                       ,x-int((BBOX_LENGTH-1)/2):x+int((BBOX_LENGTH-1)/2)])
                
            if len(pair) == 2:
                patch.append(np.array(pair))
                match.append(r)
                
        while len(patch) < NUM_MATCHES:
            patch.append(np.zeros((2,BBOX_LENGTH-1,BBOX_LENGTH-1, IMG_CHN)))
            match.append(None)
            
        patch = np.array(patch)
        patches.append(patch)
        match = np.array(match)
        matches.append(match)

        
    patches = np.array(patches)
    matches = np.array(matches)   

    return [patches[:,:,0,:,:,:], patches[:,:,1,:,:,:], matches]
    

def get_match_info(refs,snss):
    """
    returns in 255 scale
    """
    kprs,kpss, drs, dss = extract_feature_coor_batch(refs,snss)
    p_ref, p_sns, matches =  extract_match_patches(refs, snss, kprs, kpss, drs, dss)
    return [p_ref, p_sns, matches, kprs, kpss]



In [11]:
def generate_generator_multiple(generator, path, batch_size = 16, img_height = IMG_HEIGHT, img_width = IMG_WIDTH):

        gen_ref = generator.flow_from_directory(path,
                                              classes = ["ref"],
                                              target_size = (img_height,img_width),
                                              batch_size = batch_size,
                                              shuffle=False, 
                                              seed=7)

        gen_sns = generator.flow_from_directory(path,
                                              classes = ["sns"],
                                              target_size = (img_height,img_width),
                                              batch_size = batch_size,
                                              shuffle=False, 
                                              seed=7)
        while True:
                X1i = gen_ref.next()
                X2i = gen_sns.next()
                x,y,matches, _, _ = get_model_inputs(X1i[0].astype(np.uint8), X2i[0].astype(np.uint8))
                
                yield [ X1i[0].astype(np.uint8), X2i[0].astype(np.uint8) ], x #Yield both images and their mutual label
                
class Dataloader:
    def __init__(self, train_path, test_path, batch_size = 16):
        
        train_imgen = keras.preprocessing.image.ImageDataGenerator()
        test_imgen = keras.preprocessing.image.ImageDataGenerator()

        self.train_generator = generate_generator_multiple(generator=train_imgen,
                                               path = str(train_path),
                                               batch_size=batch_size)       

        self.test_generator = generate_generator_multiple(test_imgen,
                                              path = str(test_path),
                                              batch_size=batch_size)              

        
    def load_image(self, test = False):
        if test:
            return next(self.test_generator)[0]
        else:
            return next(self.train_generator)[0]
    
    def load_dl(self):
        return [self.train_generator, self.test_generator]
    
    
def get_model_inputs(refs,sns):
    p_ref, p_sns, matches, kprs, kpss = get_match_info(refs, sns)
    p_ref = (p_ref.astype(np.float32) / 255.0 ).reshape((p_ref.shape[0]*p_ref.shape[1],\
                                                        p_ref.shape[2], p_ref.shape[3], p_ref.shape[4]))
    p_sns = (p_sns.astype(np.float32) / 255.0).reshape((p_sns.shape[0]*p_sns.shape[1],\
                                                        p_sns.shape[2], p_sns.shape[3], p_sns.shape[4]))
    #matches = matches.reshape(matches.shape[0] * matches.shape[1])
    return [p_ref, p_sns, matches, kprs, kpss]
        
def visualize_corresponding_patches(p1, p2):
    for j in range(50):
        vis = (np.concatenate((p1[0][j], p2[0][j]), axis=1))
        cv2.imwrite("patch pair {}.jpg".format(j), vis)
        
def visualize_coords(img, c):
    for j in range(500):
        cv2.circle(img[0], (c[0][j][0], c[0][j][1]) , 1, (0, 0, 255), -1)
    cv2.imwrite("New feture img.jpg", img[0])
        
'''    
def test_loader():
    dl = Dataloader("./test_data","./test_data")
    x,y,matches,imgs= dl.load_image()
    refs, sns = [] , []
    for r,s in imgs:
        refs.append(r)
        sns.append(s)
    refs = np.array(refs)
    sns = np.array(sns)
    print(refs.shape)
    
    p1,p2,c1,c2 = get_match_info(refs,sns)
    visualize_corresponding_patches(p1, p2)
    visualize_coords(refs,c1)
    
    p1,p2,matches, kprs,kpss = get_model_inputs(refs,sns)
    print(p1.shape)
    print(matches.shape)
    print(kprs.shape)
'''

'    \ndef test_loader():\n    dl = Dataloader("./test_data","./test_data")\n    x,y,matches,imgs= dl.load_image()\n    refs, sns = [] , []\n    for r,s in imgs:\n        refs.append(r)\n        sns.append(s)\n    refs = np.array(refs)\n    sns = np.array(sns)\n    print(refs.shape)\n    \n    p1,p2,c1,c2 = get_match_info(refs,sns)\n    visualize_corresponding_patches(p1, p2)\n    visualize_coords(refs,c1)\n    \n    p1,p2,matches, kprs,kpss = get_model_inputs(refs,sns)\n    print(p1.shape)\n    print(matches.shape)\n    print(kprs.shape)\n'

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def feature_extractor_model():
    feature_extractor_input = keras.Input(shape=( (2*int((BBOX_LENGTH-1)/2))**2*IMG_CHN, ) )
    print(feature_extractor_input.shape)
    x = layers.Flatten()(feature_extractor_input)
    x = layers.Dense(128 ,activation='relu',kernel_initializer= keras.initializers.he_normal())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation='relu',kernel_initializer= keras.initializers.he_normal())(x)
    x = layers.BatchNormalization()(x)
    feature_extractor_output = layers.Dropout(0.2)(x)
    feature_extractor = keras.Model(feature_extractor_input,feature_extractor_output, name='feature_extractor')
    feature_extractor.summary()
    return feature_extractor

def classifier_model():
    classification_input = keras.Input(shape=( 128, ) )
    x = layers.Dense(64 ,activation='relu',kernel_initializer = keras.initializers.he_normal())(classification_input)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    classification_output = layers.Dense(1 ,activation='softmax',kernel_initializer=keras.initializers.he_normal())(x)
    classifier = keras.Model(classification_input,classification_output, name='classifier')
    classifier.summary()
    return classifier

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

def patch_dist(p1,p2):
    return np.mean(((p1 - p2)**2)**0.5)

def get_central_coor(patch,img):
    W,H = patch.shape[0], patch.shape[1]
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            print("{} {}".format(i,j))
            if i+W < img.shape[0] and j+H < img.shape[1] and patch_dist(img[i:i+W, j:j+H, :],patch) < 1:
                return (i+W/2, j + H/2)
            
    print("patch does not exist in img.")
    return None

class PatchRanker(keras.Model):
    def __init__(self, name = "patch_ranker", **kwargs):
        super(PatchRanker, self).__init__(name=name, **kwargs)
        self.feature_extractor = feature_extractor_model()
        self.classifier = classifier_model()
        
    def call(self, inputs):
        zipped = inputs
        refs, sns = [] ,[]
        for r,s in imgs:
            refs.append(r)
            sns.append(s)
        refs = np.array(refs)
        sns = np.array(sns)
        f_x = self.feature_extractor(x)
        f_y = self.feature_extractor(y)
        f_xy = tf.concat([f_x,f_y],1)
        classified = self.classifier(f_xy)
        pred = tf.reshape(classified, [-1, NUM_MATCHES, 1])
        
        inds = tf.argsort(pred, axis = 1)
        selected_inds = tf.cast(tf.where(inds < 50, tf.ones_like(inds), tf.zeros_like(inds)), tf.float32 )[:,:,0] # 6*500*1
        selected_matches = np.where( selected_inds.numpy() == 1.0, matches, np.zeros_like(matches)  )
        good_matches = []
        for i in range(selected_matches.shape[0]):
            good_matches.append(selected_matches[i][selected_matches[i] != 0])

        kprs, kpss, _, _ =  extract_feature_batch(refs,snss)
        homo = calc_homographies(kprs, kpss, good_matches)
        imgs = np.array(register_images(snss, homo))

        for i in range(6):
            cv2.imwrite("registered_cv2{}.jpg".format(i), imgs[i])
        print("registered!!")
        _, _, matches, kp1s, kp2s = get_match_info(refs, imgs)

        feature_diss = []
        coor_diss = []
        
        
        for i in range(kp1s.shape[0]):
            coor_dis = 0
            feature_dis = 0
            valid_count = 0
            for r in matches[i]:
                if r is None:
                    continue
                valid_count+=1
                x1, y1, _ = kp1s[i][r.queryIdx]
                x2, y2, _ = kp1s[i][r.trainIdx]
                #print("({},{})".format(x1,y1))
                #print("({},{})".format(x2,y2))
                coor_dis += ((x1-x2)**2 + (y1-y2)**2)**0.5 #euc dist

                feature_dis += r.distance
    
            coor_diss.append(coor_dis/valid_count)
            feature_diss.append(feature_dis/valid_count)
        
        coor_diss = np.array(coor_diss)
        feature_diss = np.array(feature_diss)
    
        
        gt = []
        for c_d, f_d in zip(coor_diss, feature_diss):
            if (c_d < DIST_THRES and f_d < DIST_THRES):
                gt.append(np.ones_like(pred[0]))
            else:
                gt.append(np.zeros_like(pred[0]))
        gt = np.array(gt)
        loss = np.sum((gt - pred)**2)
        self.add_loss(loss)

        return imgs


In [21]:
#create DNN model

DIST_THRES = 20
lr = 1e-3
batch_size = 6
trainset_size = 6
testset_size = 6
epochs = 100

patch_ranker = PatchRanker()
optimizer = keras.optimizers.Adam(learning_rate=lr)
patch_ranker.compile(optimizer = optimizer)

dl = Dataloader(train_path = "./test_data", test_path= "./test_data", batch_size = batch_size)
#refs, sns = dl.load_image()
#imgs = patch_ranker([refs,sns])
#dl = Dataloader(train_path = "./test_data", test_path= "./test_data", batch_size = batch_size)
train_generator, test_generator = dl.load_dl()
patch_ranker.fit_generator(train_generator,
                                steps_per_epoch=trainset_size/batch_size,
                                epochs = epochs,
                                validation_data = test_generator,
                                validation_steps = testset_size/batch_size,
                                use_multiprocessing = True,
                                shuffle=False)
